<a href="https://colab.research.google.com/github/mmsamiei/final-ms-thesis-code/blob/master/all_here.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Sun Sep 13 14:37:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# Knowledge Selection


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
!pip -q install transformers
!pip -q install wikipedia

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.2MB/s 
     |████████████████████████████████| 890kB 15.5MB/s 
     |████████████████████████████████| 3.0MB 27.8MB/s 


In [6]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
import wikipedia

random_state = 313

tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-6_H-256_A-4')

In [7]:
import random
import pandas as pd
import logging

class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, csv_file, frac=1, split_rate=1, shift=0, max_len=512):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.dialogues = pd.read_csv(csv_file)
        self.dialogues.dropna(inplace=True)
        
        #self.dialogues = self.dialogues.sample(frac=1, random_state=random_state).reset_index(drop=True)

        self.dialogues = self.dialogues[self.dialogues.index % split_rate == 0]
        
        # s = self.dialogues['history'].apply(len).sort_values().index
        # self.dialogues = self.dialogues.reindex(s)
        # s = self.dialogues['true_sentence'].apply(len).sort_values().index
        # self.dialogues = self.dialogues.reindex(s)

        self.dialogues = self.dialogues.reindex(index=np.roll(self.dialogues.index,shift))


        self.dialogues.dropna(inplace=True)

        self.max_len = max_len


    def __len__(self):
        return len(self.dialogues)

    @staticmethod
    def truncuate_join_pair_sentence(sentence1, sentence2, max_len=510):

        """
        truncuate sentence one from head and sentence two from tail
        Args:
            sentence1 (string): first sentence
            sentence2 (string): seconde sentence
        """
        temp1 = tokenizer.encode(sentence1,add_special_tokens=False)
        temp2 = tokenizer.encode(sentence2,add_special_tokens=False)
        ### two above line may cause warning but no problem because we've handle them below
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        seq_1 = temp1
        seq_2 = temp2
        num_tokens_to_remove = len(temp1) + len(temp2) + 3 - max_len
        if num_tokens_to_remove > 0 :
            seq_1, seq_2, _ = tokenizer.truncate_sequences(temp1[::-1],temp2, num_tokens_to_remove=num_tokens_to_remove)
            seq_1.reverse()
        result_list = [tokenizer.cls_token_id]+seq_1+[tokenizer.sep_token_id]+seq_2+[tokenizer.sep_token_id]
        token_type_ids = [0] * (len(seq_1) + 2) + [1] * (len(seq_2) + 1)
        return result_list, token_type_ids


    def __getitem__(self, idx):
      
        
        history = self.dialogues.iloc[idx].history
        true_sentence = self.dialogues.iloc[idx].true_sentence
        false_sentence = self.dialogues.iloc[idx].false_sentence


        true_pair, true_pair_segments = MyDataset.truncuate_join_pair_sentence(history, true_sentence, self.max_len)
        false_pair, false_pair_segments  = MyDataset.truncuate_join_pair_sentence(history, false_sentence, self.max_len)
        
        

        true_pair = torch.LongTensor(true_pair)
        false_pair = torch.LongTensor(false_pair)

        true_pair_segments = torch.LongTensor(true_pair_segments)
        false_pair_segments = torch.LongTensor(false_pair_segments)


        sample = {'true_pair': true_pair, 'false_pair': false_pair,
                  'true_pair_segments': true_pair_segments,
                  'false_pair_segments': false_pair_segments}

        return sample

In [9]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModel

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    self.bert = AutoModel.from_pretrained("google/bert_uncased_L-6_H-256_A-4")
    self.fc = nn.Linear(256,1)
    self.activation = nn.Tanh()
    nn.init.xavier_normal_(self.fc.weight)
  
  def forward(self, x, segments_tensors):
        temp = x
        temp = self.bert(temp, token_type_ids=segments_tensors)[0]
        ## temp = [batch, len, hid_size]
        temp = temp[:,0,:]
        temp = self.fc(temp)
        temp = self.activation(temp)
        temp = (temp - (-1))/2 
        return temp

dev = torch.device('cuda')
model = Model().to(dev)

model_dir = "/content/drive/My Drive/Final-Thesis/Phase2/Tawbah_Linear/"
checkpoint = torch.load(model_dir+'tawbah_25000steps.model')
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model.eval()

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      

#Response Generation


In [10]:
enc_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
dec_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [11]:
class MyDataset2(Dataset):
    """My dataset."""

    def __init__(self, csv_file, split_rate=1, max_len=128, shift=0, sort=True, train=False):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """

        self.train = train

        self.dialogues = pd.read_csv(csv_file)
        #self.dialogues.dropna(inplace=True)
        
        self.dialogues.fillna("", inplace=True)
        self.dialogues = self.dialogues[self.dialogues.index % split_rate == 0]

        self.dialogues.history = self.dialogues.history.str.replace(
            "[SEP]", "</s>", n=-1, regex=False) ### SPECIAL CHARACHTER FOR TURN
        

        #self.dialogues = self.dialogues.sample(frac=1, random_state=random_state).reset_index(drop=True)

        s = self.dialogues['history'].apply(len).sort_values().index
        self.dialogues = self.dialogues.reindex(s)

        s = self.dialogues['knowledge'].apply(len).sort_values().index
        self.dialogues = self.dialogues.reindex(s)

        s = self.dialogues['response'].apply(len).sort_values().index
        self.dialogues = self.dialogues.reindex(s)

        self.dialogues = self.dialogues.reindex(index=np.roll(self.dialogues.index,shift))

        self.max_len = max_len


    def __len__(self):
        return len(self.dialogues)

    @staticmethod
    def truncuate_join_pair_sentence(sentence1, sentence2, max_len=510):

        """
        truncuate sentence one from head and sentence two from tail
        Args:
            sentence1 (string): first sentence
            sentence2 (string): seconde sentence
        """
        temp1 = enc_tokenizer.encode(sentence1,add_special_tokens=False)
        temp2 = enc_tokenizer.encode(sentence2,add_special_tokens=False)
        ### two above line may cause warning but no problem because we've handle them below
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        seq_1 = temp1
        seq_2 = temp2
        num_tokens_to_remove = len(temp1) + len(temp2) + 3 - max_len
        if num_tokens_to_remove > 0 :
            seq_1, seq_2, _ = enc_tokenizer.truncate_sequences(temp1[::-1],temp2, num_tokens_to_remove=num_tokens_to_remove,
                                                               truncation_strategy='longest_first')
            seq_1.reverse()
        result_list = [enc_tokenizer.cls_token_id]+seq_1+[enc_tokenizer.sep_token_id]+[12905]+seq_2+[34437]
        token_type_ids = [0] * (len(seq_1) + 2) + [1] * (len(seq_2) + 2)
        if(len(result_list)>1000):
          print(len(result_list))

        return result_list, token_type_ids


    def __getitem__(self, idx):
      
        
        history = self.dialogues.iloc[idx].history
        knowledge = self.dialogues.iloc[idx].knowledge
        response = self.dialogues.iloc[idx].response

        # if self.train:
        #   r = random.choice(range(len(train_dataset.dialogues)))
        #   noise_knowledge = self.dialogues.iloc[r].knowledge
        #   if random.random() > 0.5:
        #     knowledge = knowledge  + noise_knowledge
        #   else:
        #     knowledge = noise_knowledge +  knowledge

        input_pair, input_pair_segments = MyDataset.truncuate_join_pair_sentence(history, knowledge, self.max_len)
                

        input_pair = torch.LongTensor(input_pair)

        input_pair_segments = torch.LongTensor(input_pair_segments)

        response_tensor = torch.LongTensor(dec_tokenizer.encode(response, truncation=True, max_length=128))

        sample = {'input_pair': input_pair,
                  'input_pair_segments': input_pair_segments,
                  'response': response_tensor}

        return sample


In [34]:
from transformers import EncoderDecoderModel, BertTokenizer
from transformers import BartForConditionalGeneration

class Model2(nn.Module):
  def __init__(self):
    super(Model2, self).__init__()

    # self.seq2seq = EncoderDecoderModel.from_encoder_decoder_pretrained(
    #     'google/bert_uncased_L-2_H-128_A-2', 'google/bert_uncased_L-6_H-128_A-2')
    
    # for p in self.seq2seq.encoder.embeddings.parameters():
    #    p.requires_grad = False
    
    # for p in self.seq2seq.decoder.bert.embeddings.parameters():
    #    p.requires_grad = False

    self.seq2seq = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

  def forward(self, encoder_input, segments_tensors, decoder_input, **kwargs):
    '''
    encoder_input = [batch_size, enc_len]
    segments_tensors = [batch_size, enc_len]
    decoder_input = [batch_size, dec_len]
    '''
    #kwargs = {'token_type_ids':segments_tensors}
    #kwargs = {}
    outputs = self.seq2seq(input_ids=encoder_input,
                           decoder_input_ids=decoder_input,
                           use_cache=False, **kwargs)[0]
    return outputs
  
  def generate(self, encoder_input, segments_tensors, **kwargs):
    ### encoder_input = [len] in int format
    ### segment_tensors = [len]
    encoder_input = encoder_input.unsqueeze(0)
    segments_tensors = segments_tensors.unsqueeze(0)
    
    model_specific_kwargs = {}
    #model_specific_kwargs['token_type_ids'] = {'token_type_ids':segments_tensors}

    generated = self.seq2seq.generate(encoder_input, decoder_start_token_id=0,
                                       eos_token_id=2, ## <s> = 0 </s> = 2
                                       **kwargs, **model_specific_kwargs)

    #### generated = [1, len]
    return generated

dev = torch.device('cuda')
model2 = Model2().to(dev)

model_addr = '/content/drive/My Drive/Final-Thesis/Phase3/Al-Mohandes/almohandes_bestval.model'
checkpoint2 = torch.load(model_addr)
model2.load_state_dict(checkpoint2['model_state_dict'])
model2.eval()

Model2(
  (seq2seq): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
        (layers): ModuleList(
          (0): EncoderLayer(
            (self_attn): Attention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm((768

#ALL

In [133]:
sentences = sent_tokenize(wikipedia.page("Canada").content)
sentences = [sentence.strip() for sentence in sentences]
# sentences.append('no_passages_used')

history = '''
 hello [SEP] hello [SEP] I think Moose is a very attractive animal. [SEP]  think they are very cute. They were called Melete or "Practice" [SEP] Where do they often live? [SEP] they are found in Canada, most of the country, northern New England and upstate New York [SEP] I love that country what do you know about it? [SEP] Canada is a country in the northern part of North America [SEP] what is its capital [SEP] it is a very beautiful country [SEP] I have asked about canada's capital!
'''.strip()

In [134]:
candidateds = ['','','']
sml_scores = [-1,-1,-1]
for false_sentence in sentences:
  false_pair_untensor, false_segment_untensor = MyDataset.truncuate_join_pair_sentence(history, false_sentence, max_len=128)
  false_pair =  torch.LongTensor(false_pair_untensor).reshape(1, -1).to(dev)
  false_segment =  torch.LongTensor(false_segment_untensor).reshape(1, -1).to(dev)
  false_sml = model(false_pair, false_segment)
  if false_sml > sml_scores[2]:
    candidateds[2] = false_sentence
    sml_scores[2] = false_sml
    if sml_scores[2] > sml_scores[1]:
      candidateds[1], candidateds[2] = candidateds[2], candidateds[1]
      sml_scores[1], sml_scores[2] = sml_scores[2], sml_scores[1]
      if sml_scores[1] > sml_scores[0]:
        candidateds[1], candidateds[0] = candidateds[0], candidateds[1]
        sml_scores[1], sml_scores[0] = sml_scores[0], sml_scores[1]

candidateds

['Canada is a country in the northern part of North America.',
 "Its ten provinces and three territories extend from the Atlantic to the Pacific and northward into the Arctic Ocean, covering 9.98 million square kilometres (3.85 million square miles), making it the world's second-largest country by total area.",
 "Its southern and western border with the United States, stretching 8,891 kilometres (5,525 mi), is the world's longest bi-national land border."]

In [131]:
text = history.replace("[SEP]", "</s>").replace("[CLS]", "</s>") + '` ' + candidateds[0] + ' ~' 

# knowledge = ''' In North America, the moose range includes almost all of Canada (excluding the arctic and Vancouver Island), most of Alaska, northern New England and upstate New York, the upper Rocky Mountains, northern Minnesota, northern Wisconsin, Michigan's Upper Peninsula, and Isle Royale in Lake Superior '''
# text = history.replace("[SEP]", "</s>").replace("[CLS]", "</s>") + '` ' + knowledge + ' ~' 


input_tns = dec_tokenizer.encode(text, return_tensors='pt').squeeze(0).to(dev)


kwargs = {'num_beams':1,
          'num_return_sequences':1,'temperature':0.7, 'max_length':50, 'early_stopping':True,
          #'no_repeat_ngram_size':2,
          #'do_sample':True, 'top_k':5
          }
dec_tokenizer.batch_decode(model2.generate(input_tns, input_tns, **kwargs))

['<s>it is a very beautiful country</s>']